In [1]:
import sys
sys.path.append('../')

import tquant as tq
from engine.globalconfig import GlobalConfig
from engine.loader.curveload import MarketDataLoader

from datetime import date
import tensorflow as tf
import pandas as pd

# Settings

In [2]:
config = GlobalConfig.from_json()
mdl = MarketDataLoader(config)

In [3]:
tq.Settings.evaluation_date

datetime.date(2023, 9, 29)

In [4]:
calendar = tq.TARGET()
daycounter = tq.DayCounter(tq.DayCounterConvention.Actual360)
evaluation_date = tq.Settings.evaluation_date

# Market

In [5]:
rates = mdl.ir_eur_curve_estr['quote'].values
times = mdl.ir_eur_curve_estr['daycount'].values/365

rates_6m = mdl.ir_eur_curve_6m['quote'].values
times_6m = mdl.ir_eur_curve_6m['daycount'].values/365

In [6]:
estr_curve = tq.RateCurve(times, rates)
eur6m_curve = tq.RateCurve(times_6m, rates_6m)

In [7]:
# instance of the hull-white model for 1M curve simulation
mean_rev = 0.02
sigma = 0.045
hw = tq.HullWhiteProcess(mean_rev, sigma, market_curve=estr_curve)
hw_kernel = tq.GaussianRateKernel1D(hw)

# simulate short rate and discount curve 
paths = int(1e6) 
r_tensor, zb_tensor, grid = hw_kernel.rate_tensor(paths,
                                                  last_grid_time=1,
                                                  time_steps=2) # evaluation_date + 1y
# Reshape the tensor and create a discount curve object
reshaped_tensor = tf.reshape(zb_tensor, shape=(zb_tensor.shape[1], paths, 1))
rate_curve_tensor = tq.DiscountCurveSimple(times, reshaped_tensor)

# Fixed coupon prices

In [8]:
d1 = calendar.advance(evaluation_date,
                         6,
                         tq.TimeUnit.Months,
                         tq.BusinessDayConvention.ModifiedFollowing)
nominal = 1e6
fixed_coupon = tq.FixedCoupon(d1,
                              nominal,
                              accrual_start_date= evaluation_date,
                              accrual_end_date= d1,
                              ref_period_start= evaluation_date,
                              ref_period_end= d1,
                              r= 0.02,
                              daycounter = daycounter)

## prezzo di un singolo coupon

In [9]:
fixed_coupon_pricer = tq.FixedCouponDiscounting(fixed_coupon)
# price single coupon
fixed_coupon_pricer.price(estr_curve, evaluation_date)

<tf.Tensor: shape=(), dtype=float64, numpy=9857.436242982803>

## prezzo dello stesso coupon con 1M curve differenti

In [10]:
# price with tensor curves
fixed_coupon_pricer.price(rate_curve_tensor, evaluation_date)

<tf.Tensor: shape=(1000000, 1), dtype=float64, numpy=
array([[ 9790.54753149],
       [ 9782.62513205],
       [ 9928.48036901],
       ...,
       [ 9767.72937344],
       [ 9874.3641776 ],
       [10022.1526553 ]])>

## prezzo dello stesso coupon con 1M curve differenti a 50 date

In [11]:
# price with tensor curves for 50 dates
date_list = [evaluation_date] *50
[fixed_coupon_pricer.price(rate_curve_tensor, d) for d in date_list]

[<tf.Tensor: shape=(1000000, 1), dtype=float64, numpy=
 array([[ 9790.54753149],
        [ 9782.62513205],
        [ 9928.48036901],
        ...,
        [ 9767.72937344],
        [ 9874.3641776 ],
        [10022.1526553 ]])>,
 <tf.Tensor: shape=(1000000, 1), dtype=float64, numpy=
 array([[ 9790.54753149],
        [ 9782.62513205],
        [ 9928.48036901],
        ...,
        [ 9767.72937344],
        [ 9874.3641776 ],
        [10022.1526553 ]])>,
 <tf.Tensor: shape=(1000000, 1), dtype=float64, numpy=
 array([[ 9790.54753149],
        [ 9782.62513205],
        [ 9928.48036901],
        ...,
        [ 9767.72937344],
        [ 9874.3641776 ],
        [10022.1526553 ]])>,
 <tf.Tensor: shape=(1000000, 1), dtype=float64, numpy=
 array([[ 9790.54753149],
        [ 9782.62513205],
        [ 9928.48036901],
        ...,
        [ 9767.72937344],
        [ 9874.3641776 ],
        [10022.1526553 ]])>,
 <tf.Tensor: shape=(1000000, 1), dtype=float64, numpy=
 array([[ 9790.54753149],
        [ 9

## AD sensitivity

In [12]:
pv, tape = fixed_coupon_pricer.price_aad(estr_curve, evaluation_date)

In [13]:
sensitivities = tape.gradient(pv, [estr_curve.rates])

In [14]:
sensy = pd.DataFrame([estr_curve.pillars.tolist(), sensitivities[0]]).T
sensy.columns = ['time', 'sensitivity']
sensy[sensy['time'] > 0.3].head(10)

,time,sensitivity
9,0.334247,NaN
10,0.419178,NaN
11,0.49589,"tf.Tensor(-4566.753499996747, shape=(), dtype=..."
12,0.583562,"tf.Tensor(-389.346388836273, shape=(), dtype=f..."
13,0.665753,NaN
14,0.747945,NaN
15,0.832877,NaN
16,0.917808,NaN
17,1.005479,NaN
18,1.084932,NaN
